In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ray
#ray.init(huge_pages=True, plasma_directory="/mnt/hugepages")
ray.init()
@ray.remote(num_cpus=1,num_gpus=0)
class ParameterServer():
	def __init__(self):
		self.parameters = dict()
		self.parameters['q'] = dict()

	def get(self):
		return self.parameters

	def put_Q_dict(self, new_q_dict):
		self.parameters['q'] = new_q_dict

2020-03-09 09:03:55,042	INFO resource_spec.py:212 -- Starting Ray with 2.0 GiB memory available for workers and up to 1.01 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-09 09:03:55,717	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


In [3]:
from actor import Actor
from learner import Learner
from replay import ReplayMemory

num_actors = 3
B = 50
periodic_update_frequency = 400
shared_experience_replay_size = 2 * 10 ** 6
num_actions = 6
env = 'PongNoFrameskip-v4'

shared_memory = ReplayMemory.remote(shared_experience_replay_size, alpha=0.6)
shared_state = ParameterServer.remote()

learner = Learner.remote(num_actions, shared_memory = shared_memory, shared_state = shared_state)

actors = [Actor.remote(i, env, shared_memory, shared_state, n=3) for i in range(num_actors)]

2020-03-09 09:04:00,931	WARNING worker.py:1058 -- The actor or task with ID fffffffffffffffff66d17ba0100 is infeasible and cannot currently be scheduled. It requires {GPU: 1.000000}, {CPU: 1.000000} for execution and {CPU: 1.000000}, {GPU: 1.000000} for placement, however there are no nodes in the cluster that can provide the requested resources. To resolve this issue, consider reducing the resource requests of this task or add nodes that can fit the task.


In [4]:
import time
time.sleep(5)

In [5]:
[a.run.remote(1000) for a in actors]#actors[0].run.remote(1000)

[ObjectID(6170691ebdfaeef644ee453c010000c801000000),
 ObjectID(cd8f5689d0aa5a397e0a4dfc010000c801000000),
 ObjectID(55c3b2b635949d816f53dca1010000c801000000)]

In [6]:
learner.learn.remote(10000)

ObjectID(9fc77bf30b438997f66d17ba010000c801000000)

In [7]:
experience_batch = ray.get(shared_memory.sample.remote(32, 0.6))
#new_priorities, loss = self.train(experience_batch)

RayTaskError(ZeroDivisionError): [36mray::ReplayMemory.sample()[39m (pid=720, ip=10.132.128.211)
  File "python/ray/_raylet.pyx", line 452, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 430, in ray._raylet.execute_task.function_executor
  File "/Users/gerty/Desktop/DQN/RL/ApeX_DQN/replay.py", line 343, in sample
    p_min = self._it_min.min() / self._it_sum.sum()
ZeroDivisionError: float division by zero

In [ ]:
device = 'cuda'
s,a,r,s_prime, gamma, qS_tpn, qS_t, key, weights, idxes = experience_batch
#s,a,r,s_prime,done_mask,weights,idxes = experience_batch

weights = torch.Tensor(weights).to(device)
s = torch.as_tensor(s).to(device)
a = torch.LongTensor(a).unsqueeze(1).to(device)
r = torch.as_tensor(r).unsqueeze(1).to(device)
s_prime = torch.as_tensor(s_prime).to(device)
gamma = torch.Tensor(gamma).unsqueeze(1).to(device)

In [ ]:
from models import Qnet
q = Qnet(84,84,4,6).to(device)
q_target = Qnet(84,84,4,6).to(device)

In [ ]:
q_out = q(s)
q_a = q_out.gather(1,a)

argmax_q = q(s_prime).argmax(1).unsqueeze(1)
q_prime = q_target(s_prime).gather(1,argmax_q)
target = r + gamma * q_prime# * done_mask
TD_errors = q_a - target
with torch.no_grad():
    new_priorities = np.abs(TD_errors.cpu()) + 10 ** -5
loss = ((TD_errors ** 2).view(-1) * weights).mean()
print(loss)

In [ ]:
ray.get(shared_memory.sample.remote(2,0.5))
ray.get(shared_memory.size.remote())

In [ ]:
import torch
import numpy as np

In [ ]:
from models import Qnet
net = Qnet(84,84,4,6)
net2 = Qnet(84,84,4,6).cuda()
data = torch.rand((1,4,84,84))
data2 = data.cuda()

In [ ]:
%%time
with torch.no_grad():
    out = net(data).numpy()

In [ ]:
%%time
with torch.no_grad():
    out = net2(data2).cpu().numpy()

2020-03-09 09:04:16,062	WARNING worker.py:1058 -- The actor or task with ID ffffffffffffffff6f53dca10100 is pending and cannot currently be scheduled. It requires {CPU: 1.000000} for execution and {CPU: 1.000000} for placement, but this node only has remaining {node:10.132.128.211: 1.000000}, {object_store_memory: 0.683594 GiB}, {memory: 2.001953 GiB}. In total there are 0 pending tasks and 1 pending actors on this node. This is likely due to all cluster resources being claimed by actors. To resolve the issue, consider creating fewer actors or increase the resources available to this Ray cluster. You can ignore this message if this Ray cluster is expected to auto-scale.
2020-03-09 09:04:18,770	ERROR worker.py:998 -- Possible unhandled error from worker: ray::Actor.__init__() (pid=723, ip=10.132.128.211)
  File "python/ray/_raylet.pyx", line 452, in ray._raylet.execute_task
  File "python/ray/_raylet.pyx", line 430, in ray._raylet.execute_task.function_executor
  File "/Users/gerty/Desk